In [87]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

from tensorflow.python.keras.utils import tf_utils

In [88]:
class TestLayer(keras.layers.Layer):
        
    def call(self, inputs, training=None):
        if training is None:
            training = keras.backend.learning_phase()
        
        train_fn = lambda: tf.zeros(tf.shape(inputs))
        eval_fn = lambda: tf.ones(tf.shape(inputs))
            
        return tf_utils.smart_cond(training, train_fn, eval_fn)

In [67]:
keras.backend.learning_phase()

0

In [89]:
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer = TestLayer()
        
    def call(self, inputs, training=None):
        return self.layer(inputs, training)

In [90]:
model = MyModel()

In [91]:
X = np.random.rand(97,100)
y = np.ones((97,100))

In [94]:
model(X, False)

<tf.Tensor: id=878, shape=(97, 100), dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>

In [95]:
model.compile(optimizer='adam',loss='mse')

In [96]:
model.fit(x=X, y=y, epochs=5)

Epoch 1/5
97/97 [==============================] - 0s 740us/sample - loss: 1.0000
Epoch 2/5
97/97 [==============================] - 0s 72us/sample - loss: 1.0000
Epoch 3/5
97/97 [==============================] - 0s 113us/sample - loss: 1.0000
Epoch 4/5
97/97 [==============================] - 0s 72us/sample - loss: 1.0000
Epoch 5/5
97/97 [==============================] - 0s 82us/sample - loss: 1.0000


In [97]:
model.predict(X)

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [56]:
drop = keras.layers.Dropout(0.5)

In [102]:
a = np.array([[1.,2.,3.,4.,5.], [5.,4.,3.,2.,1.]])
drop(a, training=False)

<tf.Tensor: id=1415, shape=(2, 5), dtype=float64, numpy=
array([[1., 2., 3., 4., 5.],
       [5., 4., 3., 2., 1.]])>

In [70]:
inp = keras.layers.Input((5,))
out = keras.layers.Dropout(0.5)(inp)
model = keras.Model(inp, out)

In [72]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [81]:
model(a,  training=True)

<tf.Tensor: id=868, shape=(1, 5), dtype=float64, numpy=array([[0., 4., 6., 0., 0.]])>

In [103]:
norm1 = keras.layers.BatchNormalization()
norm2 = keras.layers.BatchNormalization()

In [104]:
norm1(a, training=True), norm2(a, training=True)

(<tf.Tensor: id=1481, shape=(2, 5), dtype=float64, numpy=
 array([[-0.99987502, -0.99950037,  0.        ,  0.99950037,  0.99987502],
        [ 0.99987502,  0.99950037,  0.        , -0.99950037, -0.99987502]])>,
 <tf.Tensor: id=1546, shape=(2, 5), dtype=float64, numpy=
 array([[-0.99987502, -0.99950037,  0.        ,  0.99950037,  0.99987502],
        [ 0.99987502,  0.99950037,  0.        , -0.99950037, -0.99987502]])>)

In [105]:
norm1(a, training=False)

<tf.Tensor: id=1561, shape=(2, 5), dtype=float64, numpy=
array([[0.95530577, 1.96901574, 2.9834559 , 3.96801649, 4.89471102],
       [4.89471102, 3.96801649, 2.9834559 , 1.96901574, 0.95530577]])>

In [106]:
norm2(a, training=False)

<tf.Tensor: id=1575, shape=(2, 5), dtype=float64, numpy=
array([[0.95530577, 1.96901574, 2.9834559 , 3.96801649, 4.89471102],
       [4.89471102, 3.96801649, 2.9834559 , 1.96901574, 0.95530577]])>

In [112]:
import sys
sys.path.append('../')

from layers.noisy_bn import NoisyBatchNormalization
%load_ext autoreload
%autoreload 2

In [135]:
noisy_batch = NoisyBatchNormalization(alpha=0.0)

In [136]:
noisy_batch(a, training=True)

mean, var tf.Tensor([3. 3. 3. 3. 3.], shape=(5,), dtype=float64) tf.Tensor([4. 1. 0. 1. 4.], shape=(5,), dtype=float64)
tf.Tensor(
[[-0.99987502 -0.99950037  0.          0.99950037  0.99987502]
 [ 0.99987502  0.99950037  0.         -0.99950037 -0.99987502]], shape=(2, 5), dtype=float64) tf.Tensor(
[[-0.  0. -0. -0. -0.]
 [ 0. -0.  0.  0. -0.]], shape=(2, 5), dtype=float64) tf.Tensor([[3. 3. 3. 3. 3.]], shape=(1, 5), dtype=float64) tf.Tensor([[4. 1. 0. 1. 4.]], shape=(1, 5), dtype=float64)


<tf.Tensor: id=2234, shape=(2, 5), dtype=float64, numpy=
array([[-0.99987502, -0.99950037,  0.        ,  0.99950037,  0.99987502],
       [ 0.99987502,  0.99950037,  0.        , -0.99950037, -0.99987502]])>

In [ ]:
noisy_batch(a, train_)